# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.csv')

This non-commercial license of GraphLab Create for academic use is assigned to xinningh@gmail.com and will expire on October 24, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Martin\AppData\Local\Temp\graphlab_server_1482197493.log.0


Finished parsing file C:\Users\Martin\kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.148134 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,float,long,float,long,long,float,long,long,long,long,long,long,long,long,long,float,float,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\Martin\kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.120108 secs.

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.661998     | 6869667.894269     | 385998.458664 |

| 6         | 7        | 0.000002  | 1.715547     | 6847177.772191     | 380824.456039 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
table = model_all.get('coefficients')
print table.print_rows(num_rows=18)


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 274873.055652 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53105208 |  None  |
|   sqft_living    |  None | 24.4207211037 |  None  |
| sqft_living_sqrt |  None | 350.060551005 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068028819 |  None  |
|    sqft_above    |  None | 20.0247226436 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np
l1_penalty = np.logspace(1,7, num=13)

lowest_error = float('inf')
lowest_penalty = float('inf')
for penalty in l1_penalty:
    model = graphlab.linear_regression.create(training, target = 'price', features = all_features, l2_penalty=0.0, 
                                              l1_penalty=penalty, validation_set = None, verbose = False) 
    error = sum((model.predict(validation)-validation['price'])**2)
    if error < lowest_error:
        lowest_error = error
        lowest_penalty = penalty
print lowest_penalty                            
    

10.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [ ]:
10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [10]:
model = graphlab.linear_regression.create(training, target = 'price', features = all_features, l2_penalty=0.0, 
                                              l1_penalty=10, validation_set = None, verbose = False)
table = model.get('coefficients')
print table

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4263938   |  None  |
|     bedrooms     |  None |  7936.96750632   |  None  |
| bedrooms_square  |  None |  936.993363436   |  None  |
|    bathrooms     |  None |  25409.5890254   |  None  |
|   sqft_living    |  None |  39.1151369548   |  None  |
| sqft_living_sqrt |  None |   1124.6502131   |  None  |
|     sqft_lot     |  None | 0.00348362458729 |  None  |
|  sqft_lot_sqrt   |  None |   148.25839429   |  None  |
|      floors      |  None |  21204.3353131   |  None  |
|  floors_square   |  None |  12915.5244376   |  None  |
+------------------+-------+------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [11]:
print table.print_rows(num_rows=18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4263938   |  None  |
|     bedrooms     |  None |  7936.96750632   |  None  |
| bedrooms_square  |  None |  936.993363436   |  None  |
|    bathrooms     |  None |  25409.5890254   |  None  |
|   sqft_living    |  None |  39.1151369548   |  None  |
| sqft_living_sqrt |  None |   1124.6502131   |  None  |
|     sqft_lot     |  None | 0.00348362458729 |  None  |
|  sqft_lot_sqrt   |  None |   148.25839429   |  None  |
|      floors      |  None |  21204.3353131   |  None  |
|  floors_square   |  None |  12915.5244376   |  None  |
|    waterfront    |  None |  601905.585018   |  None  |
|       view       |  None |   93312.861604   |  None  |
|    condition     |  None |  6609.03547146   |  None  |
|      grade       |  None |  6206.93998293   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [12]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [13]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [14]:
non_zero_list = list()
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target = 'price', features = all_features, l2_penalty=0.0, 
                                              l1_penalty=penalty, validation_set = None, verbose = False)
    non_zero = model['coefficients']['value'].nnz()
    non_zero_list.append(non_zero)

In [15]:
print non_zero_list

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = l1_penalty_values[-6]
l1_penalty_max = l1_penalty_values[-5]
print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [18]:
lowest_error = float('inf')
lowest_penalty = 0
for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target = 'price', features = all_features, l2_penalty=0.0, 
                                              l1_penalty=penalty, validation_set = None, verbose = False)
    error = sum((model.predict(validation)-validation['price'])**2)
    non_zero = model['coefficients']['value'].nnz()
    if non_zero == max_nonzeros:
        if error < lowest_error: 
            lowest_error = error
            lowest_penalty = penalty
            lowest_model = model
    
    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [19]:
print lowest_penalty

3448968612.16


In [20]:
lowest_model.get('coefficients').print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.191779 |  None  |
|     bedrooms     |  None |  661.72256255 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9573905 |  None  |
|   sqft_living    |  None | 32.4102220841 |  None  |
| sqft_living_sqrt |  None | 690.114775638 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42027112 |  None  |
|    sqft_above    |  None |  30.011576021 |  None  |
|  sqft_basement   |  None |

In [32]:
lowest_model.get('coefficients').show()

Canvas is accessible via web browser at the URL: http://localhost:55652/index.html
Opening Canvas in default web browser.
